In [1]:
# Load the SQL extension first (only shows once)
%load_ext sql

# Import required libraries
import os
from dotenv import load_dotenv
import urllib

# Load variables from your .env file (with UTF-8 encoding)
load_dotenv(encoding='utf-8-sig')

# Fetch variables
server = os.getenv("SERVER")
database = os.getenv("DATABASE")
driver = os.getenv("DRIVER")

# Build the connection string
params = urllib.parse.quote_plus(
    f"DRIVER={{{driver}}};SERVER={server};DATABASE={database};Trusted_Connection=yes;"
)
connection_string = f"mssql+pyodbc:///?odbc_connect={params}"

%sql {connection_string}
%config SqlMagic.displaycon = False  # Hide connection info
print("Secure connection established successfully!")

Secure connection established successfully!
